In [1]:
# Purpose: 都道府県別のホール別の日別の差枚数の平均値のTOP10をLINEに送信する
#12:35分にタスクスケジューラーをセット中
import requests  
import requests
import bs4
import time
import pandas as pd
import datetime
import urllib.parse
from bs4 import BeautifulSoup


from bs4 import BeautifulSoup
import urllib.parse
import os
import traceback
import mysql
import mysql.connector
from dotenv import load_dotenv
load_dotenv(".env")

SLOCHAN_LINE_TOKEN = os.environ.get("SLOCHAN_LINE_TOKEN")
print(SLOCHAN_LINE_TOKEN)
print('読み込み完了')
time.sleep(4)
def get_cursor():
    # MySQLに接続
    conn = mysql.connector.connect(
        host=os.environ.get("AWS_SLOMAP_RDS_HOST"),
        user=os.environ.get("AWS_SLOMAP_RDS_USER"),
        password=os.environ.get("AWS_SLOMAP_RDS_PASSWORD"),
        database=os.environ.get("AWS_SLOMAP_RDS_DATABASE"),
    )
    return conn


def post_line_text(message,SLOCHAN_LINE_TOKEN):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ SLOCHAN_LINE_TOKEN}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload)

# MySQLに接続
conn = get_cursor()
# カーソルを取得
#昨日のデータを取得
target_day_num = 1
cursor = conn.cursor()
sql = f'''SELECT *
            FROM groupby_date_hall_diffcoins 
            WHERE date = DATE_SUB(CURDATE(), INTERVAL {target_day_num} DAY)
            '''
print(sql)
cursor.execute(sql)
result = cursor.fetchall()
hall_status_df = pd.DataFrame(result, columns=[col for col in cursor.column_names])

XUH4AbvcovvtXHvfDBGN1PPyQwwTlf0qgWL1My4NyYL
読み込み完了
SELECT *
            FROM groupby_date_hall_diffcoins 
            WHERE date = DATE_SUB(CURDATE(), INTERVAL 1 DAY)
            


In [5]:
_df = hall_status_df[['prefecture','hall_name']]
_df['変換後店舗名'] = _df['hall_name']
_df

C:\Users\tsc95\AppData\Local\Temp\ipykernel_19772\2625293626.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['変換後店舗名'] = _df['hall_name']


prefecture       hall_name          変換後店舗名
0          埼玉県       アリーナ桶川坂田店       アリーナ桶川坂田店
1          埼玉県           DAS蕨店           DAS蕨店
2          埼玉県  HAP-1-NHWARABI  HAP-1-NHWARABI
3          埼玉県         TOHO川越店         TOHO川越店
4          埼玉県         アイオン入間店         アイオン入間店
..         ...             ...             ...
494       神奈川県          楽園相模原店          楽園相模原店
495       神奈川県      横須賀・馬堀マリーン      横須賀・馬堀マリーン
496       神奈川県         相模大野UNO         相模大野UNO
497       神奈川県           金時鶴見店           金時鶴見店
498       神奈川県           鶴見UNO           鶴見UNO

[499 rows x 3 columns]

In [22]:

#display(hall_status_df)
hall_status_df['hall_name'] = hall_status_df['hall_name'].map(lambda x:x.replace('　','').replace(' ','').replace('日拓','').replace('駅前','').replace('TOYO','TOYOHALL'))
for prefecture_name in ['東京都']:#'埼玉県', '千葉県', '神奈川県',
    try:
        extract_prefecture_df = hall_status_df[hall_status_df['prefecture'] == prefecture_name]
        #display(extract_prefecture_df)
        extract_prefecture_df.sort_values('ave_diffcoins', ascending=False, inplace=True)
        if prefecture_name == '東京都':
            compare_diffcoins = 50
        else:
            compare_diffcoins = 0
        extract_prefecture_df_1 = extract_prefecture_df[extract_prefecture_df['ave_diffcoins'] > compare_diffcoins]
        extract_prefecture_df_1
        yesterday = datetime.date.today() - datetime.timedelta(days=target_day_num)
        date = yesterday.strftime('%Y-%m-%d')
        emoji_list = ['🥇','🥈','🥉','⭐️','⭐️']
        for _ in range(len(extract_prefecture_df_1)-5):
            emoji_list.append("⭐️")
        #文字列 昨日を曜日も加えてstr型に変換
        week = ['月','火','水','木','金','土','日']
        date = yesterday.strftime('%m').lstrip('0') + '月' + yesterday.strftime('%d').lstrip('0') + '日' \
        + '(' + week[yesterday.weekday()] + ')'
        #print(date)

        data_text =''
        for emoji ,(i,row) in zip(emoji_list,extract_prefecture_df_1.iterrows()):
            #print(i,row)
            tenpo_name = row['hall_name']
            
            if '本店' in tenpo_name:
                tenpo_name = tenpo_name.replace('本店','')
            elif '店' in tenpo_name:
                tenpo_name = tenpo_name.replace('店','')
            else:
                pass
            sum_medal = '{:,}'.format(round(int(row['sum_diffcoins']), -2))
            if int(row['sum_diffcoins']) > 0:
                sum_medal = '+' + sum_medal + '枚'

            ave_medal = str(row['ave_diffcoins']) 
            if int(row['ave_diffcoins']) > 0:
                ave_medal = '+' + ave_medal + '枚'
            daisuu = row['win_rate'].split(')')[0].replace('台','\n').replace('(','')
            data_text += f'{emoji}{tenpo_name}\n 総差{sum_medal} 平均{ave_medal} {daisuu}\n\n'
        #print(data_text)

        output_text = f'''〈{date}速報〉\n【{prefecture_name} 平均差枚ランキング】※{compare_diffcoins}枚以上抜粋\n\n'''
        #print(output_text)
        output_text += data_text
        print(output_text)

        #文字が900文字を超えたら送信
        if len(output_text) < 900:
            post_line_text(output_text,SLOCHAN_LINE_TOKEN)
            #time.sleep(10)
        else:
            concat_text = ''
            for text_line in output_text.split('\n\n'):
                concat_text += text_line + '\n'
                if len(concat_text) > 900:
                    post_line_text(concat_text,SLOCHAN_LINE_TOKEN)
                    concat_text = ''
                    #time.sleep(10)
            post_line_text(concat_text,SLOCHAN_LINE_TOKEN)
        #break
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        time.sleep(10)
        error_text = traceback.format_exc()
        post_line_text(f'{prefecture_name} {error_text}',SLOCHAN_LINE_TOKEN)

XUH4AbvcovvtXHvfDBGN1PPyQwwTlf0qgWL1My4NyYL
読み込み完了
SELECT *
            FROM groupby_date_hall_diffcoins 
            WHERE date = DATE_SUB(CURDATE(), INTERVAL 1 DAY)
            
〈1月27日(土)速報〉
【東京都 平均差枚ランキング】※50枚以上抜粋

🥇マルハン日野
 総差+298,600枚 平均+848枚 204/352


🥈エスパス渋谷新館
 総差+195,000枚 平均+485枚 198/402


🥉マルハン蒲田駅東
 総差+171,600枚 平均+463枚 179/370


⭐️キングNo.1世田谷
 総差+84,600枚 平均+408枚 97/207


⭐️マルハン大山
 総差+92,600枚 平均+406枚 112/228


⭐️マルハン池袋
 総差+110,500枚 平均+394枚 137/280


⭐️TOYOHALL
 総差+41,500枚 平均+360枚 46/115


⭐️オーパ阿佐ヶ谷
 総差+31,400枚 平均+352枚 33/89


⭐️エスパス新宿歌舞伎町
 総差+218,300枚 平均+336枚 326/648


⭐️秋葉原UNO
 総差+66,100枚 平均+335枚 86/197


⭐️メッセ昭島
 総差+121,200枚 平均+322枚 193/376


⭐️エスパス上野本館
 総差+112,100枚 平均+317枚 168/353


⭐️有楽町DUO
 総差+86,500枚 平均+301枚 127/287


⭐️ビックディッパー大井町
 総差+69,200枚 平均+288枚 110/240


⭐️カレイド経堂
 総差+26,200枚 平均+284枚 44/92


⭐️清瀬UNO
 総差+54,600枚 平均+284枚 79/192


⭐️マルハン昭島
 総差+76,200枚 平均+284枚 135/268


⭐️楽園池袋
 総差+120,800枚 平均+280枚 207/430


⭐️メッセ西葛西
 総差+118,500枚 平均+280枚 226/422


⭐️DELFEEL
 総差+22,500枚 平均+

C:\Users\tsc95\AppData\Local\Temp\ipykernel_20484\581108634.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract_prefecture_df.sort_values('ave_diffcoins', ascending=False, inplace=True)


In [21]:
#文字が900文字を超えたら送信
if len(output_text) < 900:
    post_line_text(output_text,SLOCHAN_LINE_TOKEN)
    #time.sleep(10)
else:
    concat_text = ''
    for text_line in output_text.split('\n\n'):
        concat_text += text_line + '\n'
        if len(concat_text) > 900:
            post_line_text(concat_text,SLOCHAN_LINE_TOKEN)
            concat_text = ''
            #time.sleep(10)

65

In [14]:
extract_prefecture_df_1['hall_name'].unique()

array(['マルハン日野店', 'エスパス渋谷新館', 'マルハン蒲田駅東店', 'キングNo.1世田谷店', 'マルハン大山店',
       'マルハン池袋', 'TOYO', 'オーパ阿佐ヶ谷', 'エスパス新宿歌舞伎町店', '秋葉原UNO', 'メッセ昭島店',
       'エスパス上野本館', '有楽町DUO', 'ビックディッパー大井町店', 'カレイド経堂店', '清瀬UNO',
       'マルハン昭島店', '楽園池袋店', 'メッセ西葛西店', 'DELFEEL', 'オータ東久留米店', 'マルハン東大和店',
       'オーパ荻窪', 'コンサートホール北千住', '楽園アメ横店', 'ライオン清瀬店', 'ジャンジャンマールゴット調布国領',
       'オリエント秋川店', '新橋UNO', 'マルハン青梅新町店', 'エスパス上野新館', '有楽町UNO', '飯田橋プレサス',
       'メッセ笹塚店', 'パラッツォ秋津店', 'ピーアーク北千住SSS', 'ビックディッパー花小金井店', 'メッセ三鷹店',
       'フルハウス中延店', 'メガガイア調布店', 'マルハン池袋店', '仙川UNO', 'パーラーアサヒ森下店',
       'グランドホール西高島平店', '大井ニュー東京東口店', 'メガガイア明大前店', '大山オーシャン', 'ジャラン谷在家店',
       'ビックディッパー門前仲町店', 'みとや水道橋店', 'エスパス渋谷本館', 'レッドロック学芸大学店', 'SANCO梅島店',
       'エスパス1300新小岩北口店', 'ガイア経堂店', 'メッセ高円寺店', 'エスパス新大久保店', 'ウエスタン葛西店',
       'ピーアーク竹ノ塚スタジオ', 'キューデンアネックス日暮里店', 'アイランド秋葉原店', 'ガイア大島店',
       'マルハン新宿東宝ビル店'], dtype=object)

In [ ]:
output_text.split('\n')

In [5]:
concat_text = ''
#文字が1000文字を超えたら送信
for text_line in output_text.split('\n'):
    concat_text += text_line + '\n'
    if len(concat_text) > 900:
        post_line_text(concat_text,SLOCHAN_LINE_TOKEN)
        concat_text = ''
        #time.sleep(10)

In [ ]:
while True:
    

In [1]:
import requests  
import requests
import bs4
import time
import pandas as pd
import datetime
import urllib.parse
from bs4 import BeautifulSoup

In [3]:

from bs4 import BeautifulSoup
import urllib.parse
import os
import traceback
import mysql
import mysql.connector
from dotenv import load_dotenv
load_dotenv(".env")

line_token = os.environ.get("LINE_TOKEN")
print(line_token)
def get_cursor():
    # MySQLに接続
    conn = mysql.connector.connect(
        host=os.environ.get("AWS_SLOMAP_RDS_HOST"),
        user=os.environ.get("AWS_SLOMAP_RDS_USER"),
        password=os.environ.get("AWS_SLOMAP_RDS_PASSWORD"),
        database=os.environ.get("AWS_SLOMAP_RDS_DATABASE"),
    )
    return conn


def post_line_text(message,line_token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ line_token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload)

XUH4AbvcovvtXHvfDBGN1PPyQwwTlf0qgWL1My4NyYL


In [5]:
conn = get_cursor()
# カーソルを取得
#昨日のデータを取得
target_day_num = 1
cursor = conn.cursor()
sql = f'''SELECT *
            FROM groupby_date_hall_diffcoins 
            WHERE date = DATE_SUB(CURDATE(), INTERVAL {target_day_num} DAY)
            '''
print(sql)
cursor.execute(sql)
result = cursor.fetchall()
hall_status_df = pd.DataFrame(result, columns=[col for col in cursor.column_names])
#display(hall_status_df)
for prefecture_name in ['埼玉県', '千葉県', '神奈川県', '東京都']:
    extract_prefecture_df = hall_status_df[hall_status_df['prefecture'] == prefecture_name]
    #display(extract_prefecture_df)
    extract_prefecture_df.sort_values('ave_diffcoins', ascending=False, inplace=True)
    if prefecture_name == '東京都':
        compare_diffcoins = 50
    else:
        compare_diffcoins = 0
    extract_prefecture_df_1 = extract_prefecture_df[extract_prefecture_df['ave_diffcoins'] > 0]
    extract_prefecture_df_1
    yesterday = datetime.date.today() - datetime.timedelta(days=target_day_num)
    date = yesterday.strftime('%Y-%m-%d')
    emoji_list = ['🥇','🥈','🥉','⭐️','⭐️']
    for _ in range(len(extract_prefecture_df_1)-5):
        emoji_list.append("⭐️")
    #文字列 昨日を曜日も加えてstr型に変換
    week = ['月','火','水','木','金','土','日']
    date = yesterday.strftime('%m').lstrip('0') + '月' + yesterday.strftime('%d').lstrip('0') + '日' \
    + '(' + week[yesterday.weekday()] + ')'
    #print(date)

    data_text =''
    for emoji ,(i,row) in zip(emoji_list,extract_prefecture_df_1.iterrows()):
        #print(i,row)
        tenpo_name = row['hall_name']
        
        if '本店' in tenpo_name:
            tenpo_name = tenpo_name.replace('本店','')
        elif '店' in tenpo_name:
            tenpo_name = tenpo_name.replace('店','')
        else:
            pass
        sum_medal = '{:,}'.format(round(int(row['sum_diffcoins']), -2))
        if int(row['sum_diffcoins']) > 0:
            sum_medal = '+' + sum_medal + '枚'

        ave_medal = str(row['ave_diffcoins']) 
        if int(row['ave_diffcoins']) > 0:
            ave_medal = '+' + ave_medal + '枚'
        daisuu = row['win_rate'].split(')')[0].replace('台','\n').replace('(','')
        
        data_text += f'{emoji}{tenpo_name}\n 総差{sum_medal} 平均{ave_medal} {daisuu}\n'
    #print(data_text)

    output_text = f'''〈{date}速報〉\n【{prefecture_name} 平均差枚ランキング】※{compare_diffcoins}枚以上抜粋\n\n'''
    #print(output_text)
    output_text += data_text
    print(output_text)
    post_line_text(output_text,line_token)
    time.sleep(1)
    #break

SELECT *
            FROM groupby_date_hall_diffcoins 
            WHERE date = DATE_SUB(CURDATE(), INTERVAL 1 DAY)
            
〈1月24日(水)速報〉
【埼玉県 平均差枚ランキング】※0枚以上抜粋

🥇ジャムフレンドクラブ朝霞
 総差+36,600枚 平均+153枚 94/238

🥈マルハン所沢
 総差+29,600枚 平均+95枚 117/310

🥉北上尾UNO
 総差+14,600枚 平均+77枚 45/188

⭐️コンサートホール上尾
 総差+8,100枚 平均+67枚 31/120

⭐️R7
 総差+6,500枚 平均+58枚 33/111

⭐️ニューヨークP-1
 総差+2,600枚 平均+54枚 11/48

⭐️第一プラザ坂戸にっさい
 総差+4,200枚 平均+35枚 50/118

⭐️スカイプラザ富士見
 総差+1,900枚 平均+23枚 21/82

⭐️マルハン深谷
 総差+3,800枚 平均+21枚 57/180

⭐️将軍与野
 総差+2,100枚 平均+17枚 32/121

⭐️みずほ台UNO
 総差+3,400枚 平均+14枚 84/243

⭐️第一プラザ八潮
 総差+1,600枚 平均+7枚 86/220




C:\Users\tsc95\AppData\Local\Temp\ipykernel_5088\2483293077.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract_prefecture_df.sort_values('ave_diffcoins', ascending=False, inplace=True)


〈1月24日(水)速報〉
【千葉県 平均差枚ランキング】※0枚以上抜粋

🥇ジュラク柏
 総差+13,900枚 平均+171枚 20/81

🥈マルハン習志野
 総差+44,800枚 平均+127枚 147/352

🥉ザ・ガーデン
 総差+10,400枚 平均+113枚 35/92

⭐️アップル茂原
 総差+24,200枚 平均+100枚 50/240

⭐️メガサイバースロットパチンコ市原
 総差+10,700枚 平均+99枚 48/108

⭐️マルハン八千代緑が丘
 総差+38,700枚 平均+99枚 152/389

⭐️ハリウッド野田
 総差+15,300枚 平均+78枚 77/194

⭐️TOHO幕張
 総差+7,000枚 平均+75枚 32/92

⭐️パラッツォ新検見川
 総差+7,900枚 平均+50枚 45/157

⭐️八柱UNO
 総差+9,500枚 平均+41枚 79/229

⭐️ヴィーナス市原
 総差+7,600枚 平均+38枚 37/196

⭐️パールショップともえ小見川
 総差+2,400枚 平均+28枚 20/84

⭐️ゴールド北松戸
 総差+2,400枚 平均+25枚 25/94

⭐️パールショップともえ市原
 総差+7,400枚 平均+22枚 105/333

⭐️楽園柏
 総差+8,100枚 平均+18枚 183/447

⭐️パールショップともえ松戸
 総差+6,500枚 平均+15枚 128/428

⭐️マルハン千葉北
 総差+4,400枚 平均+11枚 145/388

⭐️ガイア藤原スロット館
 総差+2,500枚 平均+8枚 74/308

⭐️マルハン八千代東
 総差+800枚 平均+5枚 57/160

⭐️ガイアネクスト行徳
 総差+700枚 平均+2枚 91/280


〈1月24日(水)速報〉
【神奈川県 平均差枚ランキング】※0枚以上抜粋


〈1月24日(水)速報〉
【東京都 平均差枚ランキング】※50枚以上抜粋




In [115]:
output_text = scraping_yesterday_groupby_prefecture_tenpo_data('埼玉県')
print(output_text)

def post_line_image_and_text(message,image_path,line_token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ line_token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files)

def post_line_image_and_text(message,line_token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ line_token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload)

2023-07-13
〈7月13日(木)速報〉
【埼玉 平均差枚ランキングTOP5】

🥇ミリオン東武練馬13号店
 総差+80,300枚 平均+590枚 62/136台
🥈ビックディッパー門前仲町店
 総差+43,400枚 平均+308枚 58/141台
🥉エスパス1300新小岩北口駅前店
 総差+89,900枚 平均+196枚 210/458台
🎖ジャラン五反田店
 総差+57,000枚 平均+171枚 148/334台
🎖エムディー目黒
 総差+18,800枚 平均+165枚 47/114台



In [17]:
main_content = soup.find(True, class_='entry-content')

In [58]:
concat_df = pd.DataFrame(columns=[],index=[])
div_list = main_content.find_all(True, class_='')
for i,div in enumerate(div_list):
    #print(i,str(div))
    if '<div>' in str(div):
        try:
            print(div.find('a').text)
            concat_df
            tenpo_name = div.find('a').text
            _df = pd.read_html(str(div))[0]
            _df['店舗名'] = tenpo_name
            concat_df = pd.concat([concat_df,_df],axis=0) 
        except:
            pass
concat_df


ミリオン東武練馬13号店
ビックディッパー門前仲町店
エスパス1300新小岩北口駅前店
ジャラン五反田店
エムディー目黒


平均差枚    総差枚  平均G数              勝率               店舗名
0   590  80287  5220   45.6%(62/136)      ミリオン東武練馬13号店
0   308  43433  2212   41.1%(58/141)     ビックディッパー門前仲町店
0   196  89900  4444  45.9%(210/458)  エスパス1300新小岩北口駅前店
0   171  56990  5427  44.3%(148/334)          ジャラン五反田店
0   165  18758  2773   41.2%(47/114)           エムディー目黒

In [81]:
row['勝率'].split('(')[1].replace(')','')

'47/114'

🥇ミリオン東武練馬13号店
 総差+80,300枚 平均+590枚 62/136台
🥈ビックディッパー門前仲町店
 総差+43,400枚 平均+308枚 58/141台
🥉エスパス1300新小岩北口駅前店
 総差+89,900枚 平均+196枚 210/458台
🎖ジャラン五反田店
 総差+57,000枚 平均+171枚 148/334台
🎖エムディー目黒
 総差+18,800枚 平均+165枚 47/114台



In [ ]:
# 〈7月7日(金)速報〉
# 【東京 平均差枚ランキングTOP5】

# 🥇マルハン昭島
# 総差+137,400枚 平均+603枚 138/228台

# 🥈メッセ高円寺
# 総差+156,048枚 平均+470枚 163/332台

# 🥉楽園渋谷駅前
# 総差+150,693枚 平均+461枚 154/327台

# 🎖マルハン池袋
# 総差+115,100枚 平均+451枚 143/255台

# 🎖マルハン亀有
# 総差+170,000枚 平均+447枚 202/380台


In [84]:
emoji_list = ['🥇','🥈','🥉','🎖','🎖']
#文字列 昨日を曜日も加えてstr型に変換
yesterday = datetime.date.today() + datetime.timedelta(days=1)
week = ['月','火','水','木','金','土','日']
date = yesterday.strftime('%m').lstrip('0') + '月' + yesterday.strftime('%d').lstrip('0') + '日' \
+ '(' + week[yesterday.weekday()] + ')'
print(date)

data_text =''
for emoji ,(i,row) in zip(emoji_list,concat_df.iterrows()):
    #print(i,row)
    tenpo_name = row['店舗名']
    sum_medal = '{:,}'.format(round(int(row['総差枚']), -2))
    if int(row['総差枚']) > 0:
        sum_medal = '+' + sum_medal + '枚'
        
    ave_medal = str(row['平均差枚']) 
    if int(row['平均差枚']) > 0:
        ave_medal = '+' + ave_medal + '枚'
    daisuu = row['勝率'].split('(')[1].replace(')','')+ '台'
        
    data_text += f'{emoji}{tenpo_name}\n 総差{sum_medal} 平均{ave_medal} {daisuu}\n'
print(data_text)

output_text = f'''〈{date}速報〉
【{prefecture} 平均差枚ランキングTOP5】\n\n'''
#print(output_text)
output_text += data_text
print(output_text)

〈7月15日(土)速報〉
【東京都 平均差枚ランキングTOP5】

🥇ミリオン東武練馬13号店
 総差+80,300枚 平均+590枚 62/136台
🥈ビックディッパー門前仲町店
 総差+43,400枚 平均+308枚 58/141台
🥉エスパス1300新小岩北口駅前店
 総差+89,900枚 平均+196枚 210/458台
🎖ジャラン五反田店
 総差+57,000枚 平均+171枚 148/334台
🎖エムディー目黒
 総差+18,800枚 平均+165枚 47/114台



7月15日(土)


In [16]:
class="entry-content"

In [19]:
concat_df = pd.read_csv('csv/brithday_df.csv')

In [20]:
concat_df['date'] = str(today.year) +'/' + concat_df['month'].astype(str) + '/' + concat_df['day'].astype(str)
concat_df['date'] = pd.to_datetime(concat_df['date'], format='%Y/%m/%d')
concat_df

month  day              name      affiliation       date
0         1    1                モモ        アニマルマンション 2023-01-01
1         1    1  ラインハルト・ヴァン・アストレア  Re:ゼロから始める異世界生活 2023-01-01
2         1    1          才賀エレオノール         からくりサーカス 2023-01-01
3         1    1               角谷杏       ガールズ＆パンツァー 2023-01-01
4         1    1              アシリパ         ゴールデンカムイ 2023-01-01
...     ...  ...               ...              ...        ...
2197     12   31             山田一二三      ダンガンロンパシリーズ 2023-12-31
2198     12   31               黎星刻           コードギアス 2023-12-31
2199     12   31                雪泉            閃乱カグラ 2023-12-31
2200     12   31          キャロライン雷電          宇宙戦艦ヤマト 2023-12-31
2201     12   31           ヴィリディアナ     ガールズアンドパンツァー 2023-12-31

[2202 rows x 5 columns]

In [264]:



#明日の誕生日の人を抽出
#extract_df = concat_df[concat_df['day'] == '明日']

## 声優の誕生日データのスクレイピング

In [20]:
# 明日の日付を生成
import datetime
today = datetime.date.today()
tomorrow = today + datetime.timedelta(days=1)
url = f'https://sulocale.sulopachinews.com/archives/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88/{tomorrow.strftime("%m%d")}'
print(url)
birthday_dfs = pd.read_html(url)
birthday_dfs
#pd.DataFrame(columns=['month', 'day', 'name','affiliation'])
for df in birthday_dfs:
    display(df)

https://sulocale.sulopachinews.com/archives/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88/0624


0
0                   6月24日が誕生日
1  ？ 北条沙都子ほうじょうさとこ（ひぐらしのなく頃に）
2          ？ 吉備土きびと（甲鉄城のカバネリ）
3        ？ 天田乾あまだけん（ペルソナシリーズ）
4       ？ 筧至鋼かけいしごう（アルドノア・ゼロ）
5      ？ 蟹座のデスマスクきゃんさー（聖闘士星矢）

0
0       ？ 岩永哲哉いわながてつやさん 52歳 A型
1     鹿目知久かなめともひさ（魔法少女まどか☆マギカ）
2  相田ケンスケあいだけんすけ（エヴァンゲリオンシリーズ）

0
0  ？ 塩屋翼しおやよくさん 64歳 B型
1    小泉こいずみ（ひぐらしのなく頃に）

0
0  ？ 西田健にしだけんさん 77歳 B型
1           ゴーダ（攻殻機動隊）

...

0
0  ？ 鷄冠井美智子かいでんみちこさん（型） ？歳 ？型
1          倉之助くらのすけ（甲鉄城のカバネリ）
2       水鏡有子みかがみゆうこ（蒼穹のファフナー）
3               婢妖ひょう（うしおととら）

0
0  ？ 桑原由気くわはらゆうきさん 31歳 O型
1          ひまわり（サミーマスコット）
2  岸間菫きしますみれ（ハイスクール・フリート）
3          時雨しぐれ（アズールレーン）
4   メイーナ（Re:ゼロから始める異世界生活）
5        ユエ（ありふれた職業で世界最強）

0
0   ？ 辻坂茜つじさかあかねさん ？歳 A型
1  岸田稚慧きしだちえ（ガールフレンド(仮)）

0
0  ？ 米丸歩よねまるあゆみさん ？歳 ？型
1  鷺森公子さぎもりきみこ（咲-Saki-）

0   1
0 NaN NaN

In [21]:
charactor_birthday_df = pd.DataFrame(columns=[ 'キャラクター名','登場作品'])
for text in birthday_dfs[0][0][1:]:
    text = text.replace(')','）').replace('？','')
    charactor_name = text.split('（')[0].replace(' ','').replace(' ','')
    charactor_affiliation = text.split('（')[1].replace('(仮','(仮)').replace('）','')
    print(charactor_name,charactor_affiliation)
    charactor_birthday_df = pd.concat([charactor_birthday_df,pd.DataFrame({'キャラクター名':[charactor_name],'登場作品':[charactor_affiliation]})],axis=0)
    
charactor_birthday_df

北条沙都子ほうじょうさとこ ひぐらしのなく頃に
吉備土きびと 甲鉄城のカバネリ
天田乾あまだけん ペルソナシリーズ
筧至鋼かけいしごう アルドノア・ゼロ
蟹座のデスマスクきゃんさー 聖闘士星矢


キャラクター名       登場作品
0  北条沙都子ほうじょうさとこ  ひぐらしのなく頃に
0         吉備土きびと   甲鉄城のカバネリ
0       天田乾あまだけん   ペルソナシリーズ
0      筧至鋼かけいしごう   アルドノア・ゼロ
0  蟹座のデスマスクきゃんさー      聖闘士星矢

In [22]:
cv_birthday_df = pd.DataFrame(columns=[ '声優','キャラクター名','登場作品'])
for i in range(1,len(birthday_dfs)):
    temp__birthday_df = birthday_dfs[i]
    #display(temp__birthday_df)
    try:
        vc_name = temp__birthday_df.loc[0][0].lstrip('？').split(' ')[1].replace('（型）','').rstrip('さん')
        vc_name = vc_name.replace(' ','').replace(' ','')

        for n in range(1,len(temp__birthday_df)):
            text = temp__birthday_df.loc[n][0].replace(')','）').replace('？','')
            charactor_name = text.split('（')[0].replace(' ','').replace(' ','')
            charactor_affiliation = text.split('（')[1].replace('(仮','(仮)').replace('）','')
            cv_birthday_df = pd.concat([cv_birthday_df,pd.DataFrame({'声優':[vc_name],'キャラクター名':[charactor_name],'登場作品':[charactor_affiliation]})],axis=0)
        #break
    except Exception as e:
        print(e)
        continue
cv_birthday_df

'numpy.float64' object has no attribute 'lstrip'


声優        キャラクター名             登場作品
0      岩永哲哉いわながてつや    鹿目知久かなめともひさ      魔法少女まどか☆マギカ
0      岩永哲哉いわながてつや  相田ケンスケあいだけんすけ     エヴァンゲリオンシリーズ
0         塩屋翼しおやよく         小泉こいずみ        ひぐらしのなく頃に
0          西田健にしだけ            ゴーダ            攻殻機動隊
0         松重慎まつしげし         ジャガ・イモ            陰の実力者
0  綿貫竜之介わたぬきりゅうのすけ         輝宗てるむね           政宗シリーズ
0  綿貫竜之介わたぬきりゅうのすけ        テオ/ワイーヨ            七つの大罪
0  綿貫竜之介わたぬきりゅうのすけ    ツギーデ・マッケンジー            陰の実力者
0    鷄冠井美智子かいでんみちこ       倉之助くらのすけ         甲鉄城のカバネリ
0    鷄冠井美智子かいでんみちこ    水鏡有子みかがみゆうこ         蒼穹のファフナー
0    鷄冠井美智子かいでんみちこ          婢妖ひょう           うしおととら
0      桑原由気くわはらゆうき           ひまわり         サミーマスコット
0      桑原由気くわはらゆうき      岸間菫きしますみれ      ハイスクール・フリート
0      桑原由気くわはらゆうき          時雨しぐれ          アズールレーン
0      桑原由気くわはらゆうき           メイーナ  Re:ゼロから始める異世界生活
0      桑原由気くわはらゆうき             ユエ     ありふれた職業で世界最強
0       辻坂茜つじさかあかね      岸田稚慧きしだちえ       ガールフレンド(仮)
0       米丸歩よねまるあゆみ    鷺森公子さぎもりきみこ          咲-Saki-

In [23]:
url = f'https://kerokero-info.com/what_today_calendar/what-today-{tomorrow.strftime("%m%d")}-1/'
res = requests.get(url)
soup = bs4.BeautifulSoup(res.text, 'html.parser')
print(soup.title.text)

ドレミの日の由来と面白い雑学、6月24日の今日は何の日？ | 雑学.com


In [287]:
cv_birthday_df

声優         キャラクター名         登場作品
0   各務立基かがみりきさん　52歳AB型             オーガ            兄
0      平修たいらおさむさん　？歳？型           章央あきお         頭文字D
0  真木駿一まきしゅんいちさん　32歳A型         ヒークヴァール   盾の勇者の成り上がり
0   麻倉ももあさくらももさん　28歳O型     伊良子美柑いらこみかん  ハイスクール・フリート
0  水野マリコみずのまりこさん　37歳A型  エイスリン・ウィッシュアート      咲-Saki-

In [62]:
cv_birthday_df

声優        キャラクター名             登場作品
0      岩永哲哉いわながてつや    鹿目知久かなめともひさ      魔法少女まどか☆マギカ
0      岩永哲哉いわながてつや  相田ケンスケあいだけんすけ     エヴァンゲリオンシリーズ
0         塩屋翼しおやよく         小泉こいずみ        ひぐらしのなく頃に
0          西田健にしだけ            ゴーダ            攻殻機動隊
0         松重慎まつしげし         ジャガ・イモ            陰の実力者
0  綿貫竜之介わたぬきりゅうのすけ         輝宗てるむね           政宗シリーズ
0  綿貫竜之介わたぬきりゅうのすけ        テオ/ワイーヨ            七つの大罪
0  綿貫竜之介わたぬきりゅうのすけ    ツギーデ・マッケンジー            陰の実力者
0    鷄冠井美智子かいでんみちこ       倉之助くらのすけ         甲鉄城のカバネリ
0    鷄冠井美智子かいでんみちこ    水鏡有子みかがみゆうこ         蒼穹のファフナー
0    鷄冠井美智子かいでんみちこ          婢妖ひょう           うしおととら
0      桑原由気くわはらゆうき           ひまわり         サミーマスコット
0      桑原由気くわはらゆうき      岸間菫きしますみれ      ハイスクール・フリート
0      桑原由気くわはらゆうき          時雨しぐれ          アズールレーン
0      桑原由気くわはらゆうき           メイーナ  Re:ゼロから始める異世界生活
0      桑原由気くわはらゆうき             ユエ     ありふれた職業で世界最強
0       辻坂茜つじさかあかね      岸田稚慧きしだちえ       ガールフレンド(仮)
0       米丸歩よねまるあゆみ    鷺森公子さぎもりきみこ          咲-Saki-

In [271]:
index_number+1 == len(cv_birthday_df)

False

In [ ]:
from PIL import Image,ImageFont,ImageDraw


def get_concat_h_multi_resize(im_list, resample=Image.BICUBIC):
    min_height = min(im.height for im in im_list)
    im_list_resize = [im.resize((int(im.width * min_height / im.height), min_height),resample=resample)
                    for im in im_list]
    total_width = sum(im.width for im in im_list_resize)
    dst = Image.new('RGB', (total_width, min_height))
    pos_x = 0
    for im in im_list_resize:
        dst.paste(im, (pos_x, 0))
        pos_x += im.width
    return dst

def get_concat_v_multi_resize(im_list, resample=Image.BICUBIC):
    min_width = min(im.width for im in im_list)
    im_list_resize = [im.resize((min_width, int(im.height * min_width / im.width)),resample=resample)
                    for im in im_list]
    total_height = sum(im.height for im in im_list_resize)
    dst = Image.new('RGB', (min_width, total_height))
    pos_y = 0
    for im in im_list_resize:
        dst.paste(im, (0, pos_y))
        pos_y += im.height
    return dst

def create_charactor_anime_cell_image(_df,image_name):
    global create_df_cell_image_path
    width_concat_lists = []
    font = ImageFont.truetype('font/NotoSansJP-Black.otf', 20)
    df_columns_list = list(_df.columns)
    cell_height = 40
    for column_number in range(len(_df.columns)):
        height_concat_lists = []
        #print(column_number)
        #print(df_columns_list[column_number])
        if df_columns_list[column_number] == 'キャラクター名':
            
            cell_width = 500
        elif df_columns_list[column_number] == '登場作品':
            cell_width = 500
        elif df_columns_list[column_number] == '声優':
            font = ImageFont.truetype('font/NotoSansJP-Black.otf', 38)
            cell_width = 500
            cell_height = 60
        elif df_columns_list[column_number] == '何の日？':
            font = ImageFont.truetype('font/NotoSansJP-Black.otf', 13)
            cell_width = 500
            cell_height = 25
        elif df_columns_list[column_number] == '日付':
            cell_width = 100
            cell_height = 25
        else:
            cell_width = 300
            font = ImageFont.truetype('font/NotoSansJP-Black.otf', 20)
            cell_height = 40
        #print(cell_width,cell_height)
        im = Image.new('RGB', (cell_width, cell_height), (0, 0, 200))  # イメージオブジェクトの生成(黒のベタ画像)
        draw = ImageDraw.Draw(im)  # Drawオブジェクトを生成  
        # フォントの指定(メイリオ48pt)
        w, h = im.size
        draw.multiline_text((cell_width/2,cell_height/2), df_columns_list[column_number], fill=(255,255,255), font=font, align ="center",anchor="mm") # 文字の描画
        w, h = im.size
        draw.rectangle((0, 0, w-1, h-1), outline = (255,255,255))
        height_concat_lists.append(im)
        name = '' #cv_birthday_df['声優'].iloc[0]
        count = 1
        for index_number ,(i,record) in enumerate(_df.iterrows()):
            #print('df_columns_list[column_number]',df_columns_list[column_number])
            if index_number == 0 and df_columns_list[column_number] == '声優':
                name = f'{record[column_number]}'
                continue
            if df_columns_list[column_number] == '声優':
                print('name',name)
                print('record[column_number',record[column_number])
                font = ImageFont.truetype('font/NotoSansJP-Black.otf', 32)
                if name == f'{record[column_number]}':
                    print('同じ名前',count)
                    count += 1
                else:
                    print('違う名前',count)
                    print((count*cell_height))
                    im = Image.new('RGB', (cell_width, (count*cell_height)), (255, 255, 255))
                    draw = ImageDraw.Draw(im) 
                    w, h = im.size
                    draw.multiline_text(((cell_width)/2,(count*cell_height)/2), f'{name}', fill=(0,0,0), font=font,anchor="mm",align ="center")
                    draw.rectangle((0, 0, w-1, h-1), outline = (0,0,0))
                    height_concat_lists.append(im)
                    count = 1
                name = f'{record[column_number]}'
                if index_number+1 == len(cv_birthday_df):
                    im = Image.new('RGB', (cell_width, (count*cell_height)), (255, 255, 255))
                    draw = ImageDraw.Draw(im) 
                    w, h = im.size
                    draw.multiline_text(((cell_width)/2,(count*cell_height)/2), f'{name}', fill=(0,0,0), font=font,anchor="mm",align ="center")
                    draw.rectangle((0, 0, w-1, h-1), outline = (0,0,0))
                    height_concat_lists.append(im)
                    count = 1
            elif df_columns_list[column_number] == '何の日？':
                im = Image.new('RGB', (cell_width, cell_height), (221, 255, 255)) 
                draw = ImageDraw.Draw(im)  # Drawオブジェクトを生成  
                font = ImageFont.truetype('font/NotoSansJP-Black.otf', 13)
                draw.multiline_text((cell_width/2,cell_height/2), f'{record[column_number]}', fill=(0,0,0), font=font,anchor="mm",align ="center") 
                w, h = im.size
                draw.rectangle((0, 0, w-1, h-1), outline = (0,0,0))
                height_concat_lists.append(im)# イメージオブジェクトの生成(黒のベタ画像)

            else:
                if (index_number + 1 ) %  2 != 0:
                    im = Image.new('RGB', (cell_width, cell_height), (255, 255, 255))  # イメージオブジェクトの生成(黒のベタ画像)
                else:
                    im = Image.new('RGB', (cell_width, cell_height), (221, 255, 255))  # イメージオブジェクトの生成(黒のベタ画像)
                draw = ImageDraw.Draw(im)  # Drawオブジェクトを生成  
                
                draw.multiline_text((cell_width/2,cell_height/2), f'{record[column_number]}', fill=(0,0,0), font=font,anchor="mm",align ="center") 
                w, h = im.size
                draw.rectangle((0, 0, w-1, h-1), outline = (0,0,0))
                height_concat_lists.append(im)
            
        #break
        concat_image_path  = rf"image/temp_image/complted_cell_{column_number}.png"
        get_concat_v_multi_resize(height_concat_lists).save(concat_image_path)
        concat_im = Image.open(concat_image_path)
        width_concat_lists.append(concat_im)
    create_df_cell_image_path = rf"image/temp_image/temp_complted_df_image_cell_{image_name}.png"
    get_concat_h_multi_resize(width_concat_lists).save(create_df_cell_image_path)
    return create_df_cell_image_path

completed_height_concat_lists = []
concat_im = Image.open('image/header_image.png')
completed_height_concat_lists.append(concat_im)

concat_image_path = create_charactor_anime_cell_image(extract_universary_df,'テスト3')
concat_im = Image.open(concat_image_path)
completed_height_concat_lists.append(concat_im)

concat_image_path = create_charactor_anime_cell_image(charactor_birthday_df,'テスト1')
concat_im = Image.open(concat_image_path)
completed_height_concat_lists.append(concat_im)

concat_image_path = create_charactor_anime_cell_image(cv_birthday_df,'テスト2')
concat_im = Image.open(concat_image_path)
completed_height_concat_lists.append(concat_im)

concat_image_path  = rf"image/temp_image/complted_image_1.png"
get_concat_v_multi_resize(completed_height_concat_lists).save(concat_image_path)

img = Image.open(concat_image_path)  # イメージを開く
print("元の画像サイズ　width: {}, height: {}".format(img.size[0], img.size[1]))  # 元の画像のサイズ出力
# 画像を指定したサイズに変更
img_resize = img.resize((1500, 2000))  # 画像のリサイズ
print("指定サイズ　width: {}, height: {}".format(img_resize.size[0], img_resize.size[1]))  # 画像のサイズ出力
img_resize.save(concat_image_path)  

In [49]:
extract_universary_df['日付'].iloc[0]

'6月24日'

ひぐらしのなく頃に


キャラクター名       登場作品
0  北条沙都子ほうじょうさとこ  ひぐらしのなく頃に

甲鉄城のカバネリ


キャラクター名      登場作品
0  吉備土きびと  甲鉄城のカバネリ

ペルソナシリーズ


...

アルドノア・ゼロ


キャラクター名      登場作品
0  筧至鋼かけいしごう  アルドノア・ゼロ

聖闘士星矢


キャラクター名   登場作品
0  蟹座のデスマスクきゃんさー  聖闘士星矢

キャラクターの誕生日一覧

ひぐらしのなく頃に
・北条沙都子ほうじょうさとこ
甲鉄城のカバネリ
・吉備土きびと
ペルソナシリーズ
・天田乾あまだけん
アルドノア・ゼロ
・筧至鋼かけいしごう
聖闘士星矢
・蟹座のデスマスクきゃんさー


In [68]:
cv_birthday_df

声優        キャラクター名             登場作品
0      岩永哲哉いわながてつや    鹿目知久かなめともひさ      魔法少女まどか☆マギカ
0      岩永哲哉いわながてつや  相田ケンスケあいだけんすけ     エヴァンゲリオンシリーズ
0         塩屋翼しおやよく         小泉こいずみ        ひぐらしのなく頃に
0          西田健にしだけ            ゴーダ            攻殻機動隊
0         松重慎まつしげし         ジャガ・イモ            陰の実力者
0  綿貫竜之介わたぬきりゅうのすけ         輝宗てるむね           政宗シリーズ
0  綿貫竜之介わたぬきりゅうのすけ        テオ/ワイーヨ            七つの大罪
0  綿貫竜之介わたぬきりゅうのすけ    ツギーデ・マッケンジー            陰の実力者
0    鷄冠井美智子かいでんみちこ       倉之助くらのすけ         甲鉄城のカバネリ
0    鷄冠井美智子かいでんみちこ    水鏡有子みかがみゆうこ         蒼穹のファフナー
0    鷄冠井美智子かいでんみちこ          婢妖ひょう           うしおととら
0      桑原由気くわはらゆうき           ひまわり         サミーマスコット
0      桑原由気くわはらゆうき      岸間菫きしますみれ      ハイスクール・フリート
0      桑原由気くわはらゆうき          時雨しぐれ          アズールレーン
0      桑原由気くわはらゆうき           メイーナ  Re:ゼロから始める異世界生活
0      桑原由気くわはらゆうき             ユエ     ありふれた職業で世界最強
0       辻坂茜つじさかあかね      岸田稚慧きしだちえ       ガールフレンド(仮)
0       米丸歩よねまるあゆみ    鷺森公子さぎもりきみこ          咲-Saki-

In [66]:
extract_df

キャラクター名   登場作品
0  蟹座のデスマスクきゃんさー  聖闘士星矢

In [76]:
print(text)

6月24日 今日は何の日？
◆キャラクターの誕生日一覧
ひぐらしのなく頃に
・北条沙都子ほうじょうさとこ
甲鉄城のカバネリ
・吉備土きびと
ペルソナシリーズ
・天田乾あまだけん
アルドノア・ゼロ
・筧至鋼かけいしごう
聖闘士星矢
・蟹座のデスマスクきゃんさー

◆声優の誕生日一覧
岩永哲哉いわながてつや
・鹿目知久かなめともひさ 魔法少女まどか☆マギカ
・相田ケンスケあいだけんすけ エヴァンゲリオンシリーズ
塩屋翼しおやよく
・小泉こいずみ ひぐらしのなく頃に
西田健にしだけ
・ゴーダ 攻殻機動隊
松重慎まつしげし
・ジャガ・イモ 陰の実力者
綿貫竜之介わたぬきりゅうのすけ
・輝宗てるむね 政宗シリーズ
・テオ/ワイーヨ 七つの大罪
・ツギーデ・マッケンジー 陰の実力者
鷄冠井美智子かいでんみちこ
・倉之助くらのすけ 甲鉄城のカバネリ
・水鏡有子みかがみゆうこ 蒼穹のファフナー
・婢妖ひょう うしおととら
桑原由気くわはらゆうき
・ひまわり サミーマスコット
・岸間菫きしますみれ ハイスクール・フリート
・時雨しぐれ アズールレーン
・メイーナ Re:ゼロから始める異世界生活
・ユエ ありふれた職業で世界最強
辻坂茜つじさかあかね
・岸田稚慧きしだちえ ガールフレンド(仮)
米丸歩よねまるあゆみ
・鷺森公子さぎもりきみこ 咲-Saki-


In [77]:
try:
    from src.utils import *
except:
    from utils import *

twitter = UtilsTwitterClass()
twitter.line_token = 'XUH4AbvcovvtXHvfDBGN1PPyQwwTlf0qgWL1My4NyYL'

読み込み完了


In [74]:
birthday = extract_universary_df['日付'].iloc[0]
text = f'{birthday} 今日は何の日？'
text += '\n◆キャラクターの誕生日一覧'
virtday_count_dict = dict(charactor_birthday_df['登場作品'].value_counts())
birthday_dict_tuple = sorted(virtday_count_dict.items(), key=lambda x:x[1], reverse=True)
for affitation in birthday_dict_tuple:
    print(affitation[0])
    extract_df = charactor_birthday_df[charactor_birthday_df['登場作品'] == affitation[0]]
    display(extract_df)
    text += f'\n{affitation[0]}'
    for index, row in extract_df.iterrows():
        text += f'\n・{row["キャラクター名"]}'
print(text) 

text += '\n\n◆声優の誕生日一覧'

for affitation in cv_birthday_df['声優'].unique():
    print(affitation[0])
    extract_df = cv_birthday_df[cv_birthday_df['声優'] == affitation]
    display(extract_df)
    text += f'\n{affitation}'
    for index, row in extract_df.iterrows():
        text += f'\n・{row["キャラクター名"]} {row["登場作品"]}'
print(text) 

ひぐらしのなく頃に


キャラクター名       登場作品
0  北条沙都子ほうじょうさとこ  ひぐらしのなく頃に

甲鉄城のカバネリ


キャラクター名      登場作品
0  吉備土きびと  甲鉄城のカバネリ

ペルソナシリーズ


...

辻


声優    キャラクター名        登場作品
0  辻坂茜つじさかあかね  岸田稚慧きしだちえ  ガールフレンド(仮)

米


声優      キャラクター名     登場作品
0  米丸歩よねまるあゆみ  鷺森公子さぎもりきみこ  咲-Saki-

6月24日 今日は何の日？
◆キャラクターの誕生日一覧
ひぐらしのなく頃に
・北条沙都子ほうじょうさとこ
甲鉄城のカバネリ
・吉備土きびと
ペルソナシリーズ
・天田乾あまだけん
アルドノア・ゼロ
・筧至鋼かけいしごう
聖闘士星矢
・蟹座のデスマスクきゃんさー

◆声優の誕生日一覧
岩永哲哉いわながてつや
・鹿目知久かなめともひさ 魔法少女まどか☆マギカ
・相田ケンスケあいだけんすけ エヴァンゲリオンシリーズ
塩屋翼しおやよく
・小泉こいずみ ひぐらしのなく頃に
西田健にしだけ
・ゴーダ 攻殻機動隊
松重慎まつしげし
・ジャガ・イモ 陰の実力者
綿貫竜之介わたぬきりゅうのすけ
・輝宗てるむね 政宗シリーズ
・テオ/ワイーヨ 七つの大罪
・ツギーデ・マッケンジー 陰の実力者
鷄冠井美智子かいでんみちこ
・倉之助くらのすけ 甲鉄城のカバネリ
・水鏡有子みかがみゆうこ 蒼穹のファフナー
・婢妖ひょう うしおととら
桑原由気くわはらゆうき
・ひまわり サミーマスコット
・岸間菫きしますみれ ハイスクール・フリート
・時雨しぐれ アズールレーン
・メイーナ Re:ゼロから始める異世界生活
・ユエ ありふれた職業で世界最強
辻坂茜つじさかあかね
・岸田稚慧きしだちえ ガールフレンド(仮)
米丸歩よねまるあゆみ
・鷺森公子さぎもりきみこ 咲-Saki-


In [7]:
output_text = ''
text_list  = 'aaaaaaaaa。bbbbbb。ccccccccccc。dddddddddddd。eeeeeeeeeee。ffffffff。gggggggggg。hhhhhhhhh。iiiiiiii。jjjjjjjjj'.split('。')
for i,text in enumerate(text_list):
    i += 1
    #print(text)
    if i > 9:
        break
    if i % 3 == 0:
        output_text += text + ',\n'
    else:
        output_text += text + ','
print(output_text)

aaaaaaaaa,bbbbbb,ccccccccccc,
dddddddddddd,eeeeeeeeeee,ffffffff,
gggggggggg,hhhhhhhhh,iiiiiiii,



In [7]:

import MeCab #Mecabインポート
wakati = MeCab.Tagger('-Owakati') #分かち書きモードのインスタンス作成

split_name_list = wakati.parse('大場ゆいおおばゆい').split()
split_name_list 

['大場', 'ゆい', 'おおば', 'ゆい']

In [13]:
import collections
c = dict(collections.Counter(split_name_list))
print(c)

{'大場': 1, 'ゆい': 2, 'おおば': 1}


In [53]:
sorted(c.items(), key=lambda x:x[1], reverse=True)

[('ゆい', 2), ('大場', 1), ('おおば', 1)]

In [17]:
'大場ゆいおおばゆい'.split('ゆい')

['大場', 'おおば', '']

In [24]:
url = 'https://netwadai.com/blog/post-2363'
universary_df  = pd.read_html(url)[0]
universary_df = universary_df.rename(columns={0:'日付',1:'何の日？'})
universary_df = universary_df[~universary_df['日付'].str.contains('記念日一覧')]
universary_df = universary_df[~universary_df['日付'].str.contains('2月29日')]
#pd.DataFrame(columns=['month', 'day', 'name','affiliation'])
display(universary_df)
tomorrow_str:str = tomorrow.strftime("%m月").lstrip('0') + tomorrow.strftime("%d日").lstrip('0') 
extract_universary_df = universary_df[universary_df['日付'].str.contains(tomorrow_str)]
extract_universary_df

日付                                               何の日？
1      1月1日  神戸港記念日。元旦。鉄腕アトムの日。初詣。太陽暦施行の日。点字新聞創刊記念日。 パブリックド...
2      1月2日                           ロケットの日。初夢。初売り・初荷・初商・書き初め
3      1月3日                                瞳の日。駆け落ちの日。戊辰戦争開戦の日
4      1月4日                                       石の日。御用始め・大発会
5      1月5日        いちごの日。囲碁の日。シンデレラの日。魚河岸初競り。つむぎの日。ホームセキュリティの日
..      ...                                                ...
372  12月26日          ボクシング・デー。プロ野球誕生の日（ジャイアンツの日）。雪印の日。聖ステファノの日
373  12月27日                                  ピーターパンの日。浅草仲見世記念日
374  12月28日          ディスクジョッキーの日。身体検査の日。仕事納め・官公庁御用納め。シネマトグラフの日
376  12月30日                         地下鉄記念日。取引所大納会。フィリピン・リサールの日
377  12月31日                       大晦日。シンデレラデー。除夜。大祓。ニューイヤーズ・イヴ

[364 rows x 2 columns]

日付              何の日？
181  6月24日  UFO記念日。ドレミの日。麦の日

['キャラクター名', '登場作品']

In [186]:
column_number = 0
df_columns_list[column_number]

'キャラクター名'

In [253]:
index_number

25

In [ ]:

for i,month in enumerate(soup.find_all(True, class_='accordion main_c')):
    #print(month)
    i += 1
    month_str :str = i 
    month.find_all('p')
    for p_text in month.find_all('p'):
        if p_text.text.replace('日','').isalnum():
            day = p_text.text.replace('日','')
        else:
            for row in p_text.text.replace(')','）').split('）'):
              try:
                  name = row.split('（')[0].replace(' ','').replace(' ','')
                  affiliation = row.split('（')[1].replace('(仮','(仮)')
                  #print(name,)
                  record = pd.DataFrame({'month': [month_str],
                      'day': [day],
                      'name': [name],
                      'affiliation':[affiliation]})
                  concat_df = pd.concat([concat_df,record],axis=0)
              except:
                  pass
    